In [0]:
%sql
create table workspace.default.orders_external12(
  order_id int,
  customer_id int,
  amount double
)
using Delta
location 's3://demodatabrics/raw/orders_external12';

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4957608752250357>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "create table workspace.default.orders_external12(\n  order_id int,\n  customer_id int,\n  amount double\n)\nusing Delta\nlocation 's3://demodatabrics/raw/orders_external12';\n")

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/s

Step 1:create Delta Table

In [0]:
data=[
    (1,"Ram",50000),
    (2,"Sita",60000),
    (3,"Arjun",55000)
]

columns=["id","name","salary"]

df=spark.createDataFrame(data,columns)
df.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  Ram| 50000|
|  2| Sita| 60000|
|  3|Arjun| 55000|
+---+-----+------+



**Step 2: Write delta table**

In [0]:
df.write.format("delta").mode("Overwrite").save("/Volumes/workspace/default/secondvoulmn/employee_delta")

Step 3:Read Delta Table

In [0]:
df_delta=spark.read.format("delta").load("/Volumes/workspace/default/secondvoulmn/employee_delta")
df_delta.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  Ram| 50000|
|  2| Sita| 60000|
|  3|Arjun| 55000|
+---+-----+------+



In [0]:
df_read=spark.read.format("delta").load("/Volumes/workspace/default/secondvoulmn/employee_delta")
df_read.show()

+---+-----+------+
| id| name|salary|
+---+-----+------+
|  1|  Ram| 50000|
|  2| Sita| 60000|
|  3|Arjun| 55000|
+---+-----+------+



**Step 3:Update using DeltaTable**

In [0]:
from delta.tables import DeltaTable

deltaTable=DeltaTable.forPath(spark,"/Volumes/workspace/default/secondvoulmn/employee_delta")
deltaTable.update(
  condition="id=1",
  set={"salary":"70000"}
)

DataFrame[num_affected_rows: bigint]

**Step 4:Display**

We can not display directly using the delta table beacuse it only for transaction 
for diplay use spark.read

In [0]:
display(spark.read.format("delta").load("/Volumes/workspace/default/secondvoulmn/employee_delta"))

id,name,salary
1,Ram,70000
2,Sita,60000
3,Arjun,55000


**step 5:Delete User**

In [0]:
deltaTable.delete("id=3")
display(spark.read.format("delta").load("/Volumes/workspace/default/secondvoulmn/employee_delta"))

id,name,salary
1,Ram,70000
2,Sita,60000


**Step 6:Acid Merge**

In [0]:
new_data=[
  (2,"Sita",65000), #Updated record
  (4,"Lakshaman",62000) #new Record
]
df_new=spark.createDataFrame(new_data,columns)
display(df_new)

id,name,salary
2,Sita,65000
4,Lakshaman,62000


In [0]:
deltaTable.alias("old") \
    .merge(
        df_new.alias("new"),
        "Old.id = new.id"
    ) \
    .whenMatchedUpdate(set={"salary":"new.salary"}) \
    .whenNotMatchedInsertAll() \
    .execute()        

In [0]:
display(spark.read.format("delta").load("/Volumes/workspace/default/secondvoulmn/employee_delta"))

id,name,salary
1,Ram,70000
2,Sita,60000


**Step 7:Time Travel Demo**

In [0]:
%sql
describe history  '/Volumes/workspace/default/secondvoulmn/employee_delta';

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
13,2025-11-20T06:17:25.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,12,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1068, p25FileSize -> 1044, numDeletionVectorsRemoved -> 1, minFileSize -> 1044, numAddedFiles -> 1, maxFileSize -> 1044, p75FileSize -> 1044, p50FileSize -> 1044, numAddedBytes -> 1044)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
12,2025-11-20T06:17:23.000Z,79028800692993,ramanshinde064@gmail.com,DELETE,"Map(predicate -> [""(id#11651L = 3)""])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,11,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1244, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 881, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 362)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
11,2025-11-20T06:17:14.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,10,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2081, p25FileSize -> 1068, numDeletionVectorsRemoved -> 1, minFileSize -> 1068, numAddedFiles -> 1, maxFileSize -> 1068, p75FileSize -> 1068, p50FileSize -> 1068, numAddedBytes -> 1068)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
10,2025-11-20T06:17:11.000Z,79028800692993,ramanshinde064@gmail.com,UPDATE,"Map(predicate -> [""(id#11267L = 1)""])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,9,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3425, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1913, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1013, rewriteTimeMs -> 1463)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
9,2025-11-20T06:16:51.000Z,79028800692993,ramanshinde064@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,8,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 1081, numDeletionVectorsRemoved -> 0, numOutputRows -> 3, numOutputBytes -> 1068)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
8,2025-11-20T05:12:25.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-043300-tjo5gyb0-v2n,7,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 3108, p25FileSize -> 1081, numDeletionVectorsRemoved -> 1, minFileSize -> 1081, numAddedFiles -> 1, maxFileSize -> 1081, p75FileSize -> 1081, p50FileSize -> 1081, numAddedBytes -> 1081)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
7,2025-11-20T05:12:23.000Z,79028800692993,ramanshinde064@gmail.com,MERGE,"Map(predicate -> [""(id#12281L = id#12284L)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1261558251566968),1120-043300-tjo5gyb0-v2n,6,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 2064, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 1, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 3633, materializeSourceTimeMs -> 325, numTargetRowsInserted -> 1, n

In [0]:
%sql
describe history delta.`/Volumes/workspace/default/secondvoulmn/employee_delta`;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
13,2025-11-20T06:17:25.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,12,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1068, p25FileSize -> 1044, numDeletionVectorsRemoved -> 1, minFileSize -> 1044, numAddedFiles -> 1, maxFileSize -> 1044, p75FileSize -> 1044, p50FileSize -> 1044, numAddedBytes -> 1044)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
12,2025-11-20T06:17:23.000Z,79028800692993,ramanshinde064@gmail.com,DELETE,"Map(predicate -> [""(id#11651L = 3)""])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,11,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1244, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 881, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 362)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
11,2025-11-20T06:17:14.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,10,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 2081, p25FileSize -> 1068, numDeletionVectorsRemoved -> 1, minFileSize -> 1068, numAddedFiles -> 1, maxFileSize -> 1068, p75FileSize -> 1068, p50FileSize -> 1068, numAddedBytes -> 1068)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
10,2025-11-20T06:17:11.000Z,79028800692993,ramanshinde064@gmail.com,UPDATE,"Map(predicate -> [""(id#11267L = 1)""])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,9,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 3425, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1913, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1013, rewriteTimeMs -> 1463)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
9,2025-11-20T06:16:51.000Z,79028800692993,ramanshinde064@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(1261558251566968),1120-060531-66dyoqs3-v2n,8,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 1081, numDeletionVectorsRemoved -> 0, numOutputRows -> 3, numOutputBytes -> 1068)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
8,2025-11-20T05:12:25.000Z,79028800692993,ramanshinde064@gmail.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(1261558251566968),1120-043300-tjo5gyb0-v2n,7,SnapshotIsolation,false,"Map(numRemovedFiles -> 3, numRemovedBytes -> 3108, p25FileSize -> 1081, numDeletionVectorsRemoved -> 1, minFileSize -> 1081, numAddedFiles -> 1, maxFileSize -> 1081, p75FileSize -> 1081, p50FileSize -> 1081, numAddedBytes -> 1081)",null,Databricks-Runtime/17.3.x-aarch64-photon-scala2.13
7,2025-11-20T05:12:23.000Z,79028800692993,ramanshinde064@gmail.com,MERGE,"Map(predicate -> [""(id#12281L = id#12284L)""], clusterBy -> [], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1261558251566968),1120-043300-tjo5gyb0-v2n,6,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 2, numTargetBytesAdded -> 2064, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 1, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 3633, materializeSourceTimeMs -> 325, numTargetRowsInserted -> 1, n

**Step 8:Access version 0**

In [0]:
df_v0=spark.read.format("delta").option("versionAsOf",0).load("/Volumes/workspace/default/secondvoulmn/employee_delta")
display(df_v0)

id,name,salary
1,Ram,50000
2,Sita,60000
3,Arjun,55000


**Step 8:View Delta Transaction Log**

In [0]:
%fs ls /Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/ 

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2859,1763613637000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000000.json,00000000000000000000.json,1869,1763613636000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000001.crc,00000000000000000001.crc,3522,1763614354000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000001.json,00000000000000000001.json,2701,1763614354000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000002.crc,00000000000000000002.crc,2884,1763614356000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000002.json,00000000000000000002.json,3001,1763614356000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000003.crc,00000000000000000003.crc,3547,1763614405000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000003.json,00000000000000000003.json,2751,1763614404000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000004.crc,00000000000000000004.crc,2884,1763614406000
dbfs:/Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000004.json,00000000000000000004.json,3026,1763614406000


# Step : fetch using the version 0 delta log

**Delta Log:-**
We look inside _delta_log to understand how Delta Lake tracks table changes using commit files. Each version describes schema, added files, removed files, and metadata — this is how ACID is implemented.

In [0]:
%fs head /Volumes/workspace/default/secondvoulmn/employee_delta/_delta_log/00000000000000000000.json

{"commitInfo":{"timestamp":1763613635314,"userId":"79028800692993","userName":"ramanshinde064@gmail.com","operation":"WRITE","operationParameters":{"mode":"Overwrite","statsOnLoad":false,"partitionBy":"[]"},"notebook":{"notebookId":"1261558251566968"},"clusterId":"1120-043300-tjo5gyb0-v2n","isolationLevel":"WriteSerializable","isBlindAppend":false,"operationMetrics":{"numFiles":"1","numRemovedFiles":"0","numRemovedBytes":"0","numDeletionVectorsRemoved":"0","numOutputRows":"3","numOutputBytes":"1068"},"tags":{"noRowsCopied":"true","restoresDeletedRows":"false"},"engineInfo":"Databricks-Runtime/17.3.x-aarch64-photon-scala2.13","txnId":"2e5420f9-326c-465f-a4ac-a1eef0454591"}}
{"metaData":{"id":"1f4c84a9-708d-4f1d-8f67-5714b0e3ef8c","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"salary\",\"

## Case 1:Failed Update -> Rollback

In [0]:
try:
   deltaTable.update(
     condition="id='ABC'",
     set={"salary":"90000"}
    )
except Exception as e:
  print("Transaction Failed: ",e)   


Transaction Failed:  [CAST_INVALID_INPUT] The value 'ABC' of the type "STRING" cannot be cast to "BIGINT" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== SQL (line 1, position 1) ==
id='ABC'
^^^^^^^^


JVM stacktrace:
org.apache.spark.SparkNumberFormatException
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidInputInCastToNumberError(QueryExecutionErrors.scala:191)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.withException(UTF8StringUtils.scala:51)
	at org.apache.spark.sql.catalyst.util.UTF8StringUtils$.toLongExact(UTF8StringUtils.scala:31)
	at org.apache.spark.sql.catalyst.expressions.Cast.$anonfun$castToLong$2(Cast.scala:974)
	at org.apache.spark.sql.catalyst.expressions.Cast.$anonfun$castToLong$2$adapted(Cast.scala:974)
	at org.apache.spark.sql.catalyst.expressions.Cast.buildCast(Cast.scala:722)
	at org.apache.spark.sql.catalyst.expre